In [33]:
from sqlalchemy import create_engine, text
import pandas as pd 
import numpy as np

In [34]:
# ======================
# PostgreSQL Config
# ======================
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(DATABASE_URL)
print("✅ Connected to PostgreSQL")

✅ Connected to PostgreSQL


In [35]:

def import_data(engine, file_path='/Users/hilmanyusoh/Desktop/Co-opDash/data/member300.csv'):
    df = pd.read_csv(file_path)
    
    # เตรียมข้อมูลวันที่
    df['birthday'] = pd.to_datetime(df['birthday'])
    df['registration_date'] = pd.to_datetime(df['registration_date'])
    df['approval_date'] = pd.to_datetime(df['approval_date'])

    with engine.begin() as conn:
        # --- นำเข้า Master Tables ---
        # 1. Branches
        pd.DataFrame({'branch_no': df['branch_no'].unique()}).to_sql('branches', conn, if_exists='append', index=False)
        
        # 2. Gender Mapping
        gender_df = pd.DataFrame({'gender_name': df['gender'].unique()})
        gender_df.to_sql('gender', conn, if_exists='append', index=False)
        gender_map = pd.read_sql("SELECT gender_no, gender_name FROM gender", conn)
        df = df.merge(gender_map, left_on='gender', right_on='gender_name')

        # 3. Careers Mapping
        career_df = pd.DataFrame({'career_name': df['career'].unique()})
        career_df.to_sql('careers', conn, if_exists='append', index=False)
        career_map = pd.read_sql("SELECT career_no, career_name FROM careers", conn)
        df = df.merge(career_map, left_on='career', right_on='career_name')

        # --- นำเข้าข้อมูลตามลำดับตาราง ---
        # 4. Members
        df[['member_id', 'gender_no', 'first_name', 'last_name', 'branch_no']].to_sql('members', conn, if_exists='append', index=False)

        # 5-9. Child Tables
        df[['member_id', 'birthday']].to_sql('birthdays', conn, if_exists='append', index=False)
        df[['member_id', 'registration_date']].to_sql('registration', conn, if_exists='append', index=False)
        df[['member_id', 'approval_date']].to_sql('approval', conn, if_exists='append', index=False)
        df[['member_id', 'career_no', 'income']].to_sql('income', conn, if_exists='append', index=False)
        
        address_cols = ['member_id', 'house_no', 'village_no', 'road', 'sub_area', 'district_area', 'province', 'postal_code']
        df[address_cols].to_sql('addresses', conn, if_exists='append', index=False)

    print("✅ นำเข้าข้อมูลลงใน 9 ตารางสำเร็จ!")
    
# --- วิธีใช้งาน ---
if __name__ == "__main__":
    import_data(engine)

✅ นำเข้าข้อมูลลงใน 9 ตารางสำเร็จ!
